# Penguin Analysis with jupyter-cc

This notebook demonstrates using `%cc` magic to work with Claude Code inside Jupyter.
Data is loaded with regular Python — `%cc` is used for analysis and visualization.

In [ ]:
%load_ext jupyter_cc

## Load Data

In [ ]:
import seaborn as sns

df = sns.load_dataset("penguins").dropna()
df.head()

## Visualize

Ask Claude to create a scatter plot from the loaded data.

In [ ]:
%cc Create a scatter plot of bill length vs depth, colored by species. Use matplotlib.

In [ ]:
# [CC] Scatter plot of bill length vs depth, colored by species
import matplotlib.pyplot as plt
from IPython.utils.capture import capture_output

with capture_output() as _claude_captured_output:
    fig, ax = plt.subplots(figsize=(8, 6))
    colors = {"Adelie": "#1f77b4", "Chinstrap": "#ff7f0e", "Gentoo": "#2ca02c"}

    for species, group in df.groupby("species"):
        ax.scatter(
            group["bill_length_mm"], group["bill_depth_mm"], label=species, color=colors[species], alpha=0.7, s=60
        )

    ax.set_xlabel("Bill Length (mm)")
    ax.set_ylabel("Bill Depth (mm)")
    ax.set_title("Bill Length vs Depth by Species")
    ax.legend(title="Species")
    plt.tight_layout()
    plt.show()

for output in _claude_captured_output.outputs:
    display(output)

## Summary Statistics

Start a fresh conversation to calculate summary stats.

In [ ]:
%cc Calculate summary statistics for body_mass_g grouped by species and sex

In [ ]:
# [CC] Summary statistics for body_mass_g grouped by species and sex
df.groupby(["species", "sex"])["body_mass_g"].agg(
    count="count",
    mean="mean",
    std="std",
    min="min",
    median="median",
    max="max",
).round(2)

## In-place Editing

Use `%cc_cur` to replace the prompt cell with generated code.

In [ ]:
# [CC] Function to get mean measurements by species
def get_species_means(species_name: str) -> dict:
    """Return mean numeric measurements for a given penguin species."""
    subset = df[df["species"] == species_name]
    if subset.empty:
        raise ValueError(f"Species '{species_name}' not found. Valid options: {df['species'].unique().tolist()}")
    return subset.mean(numeric_only=True).round(2).to_dict()


# Example usage
get_species_means("Chinstrap")